In [1]:
import json
import ray
from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env
import numpy as np
from flow.networks.ring import RingNetwork, ADDITIONAL_NET_PARAMS
from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder
from flow.core.params import SumoParams, EnvParams, InitialConfig, NetParams
from flow.core.params import VehicleParams, SumoCarFollowingParams
from flow.controllers import RLController, IDMController, ContinuousRouter
from gym.spaces.box import Box
from gym.spaces import Tuple
from flow.core.params import InFlows
from flow.controllers import SimLaneChangeController
from flow.networks import Network
import os
from flow.controllers.routing_controllers import ConstructionRouter
from flow.core.params import SumoLaneChangeParams

from env_testV2 import myEnv

ADDITIONAL_ENV_PARAMS = {
    "max_accel": 1,
    "max_decel": 1,
}

# time horizon of a single rollout
HORIZON = 1500
# number of rollouts per training iteration
N_ROLLOUTS = 20
# number of parallel workers
N_CPUS = 2

# We place one autonomous vehicle and 22 human-driven vehicles in the network
vehicles = VehicleParams()
vehicles.add("rl",
             acceleration_controller=(RLController, {}),
             lane_change_controller=(SimLaneChangeController, {}),
             routing_controller=(ConstructionRouter, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed",  
                 # we use the speed mode "obey_safe_speed" for better dynamics at the merge
             ),
             num_vehicles=2)
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             lane_change_controller=(SimLaneChangeController, {}),
             #routing_controller=(ContinuousRouter, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed",  
                 # we use the speed mode "obey_safe_speed" for better dynamics at the merge
             ),
             lane_change_params = SumoLaneChangeParams(lane_change_mode="strategic",lcpushy=1.0),
             num_vehicles=0)

# specify the edges vehicles can originate on
initial_config = InitialConfig(
    edges_distribution=["gneE4","gneE35","gneE38"]
)

# specify the routes for vehicles in the network
class Network(Network):

    def specify_routes(self, net_params):
        return {
                "gneE35": ["gneE35","gneE15","gneE13","gneE4.264","gneE4.264.110","gneE8","gneE9","gneE9.252","gneE33"],
                "gneE4": ["gneE4","gneE17","gneE11","gneE13","gneE4.264","gneE4.264.110","gneE8","gneE9","gneE9.252","gneE33"],
               "gneE8": ["gneE8","gneE9","gneE37","gneE38","gneE39","gneE4.264.110","gneE8"]
               }


inflow = InFlows()
inflow.add(veh_type="human",
           edge="gneE4",
           vehs_per_hour=1000,
            depart_lane="random",
            depart_speed="random",
            color="white")
inflow.add(veh_type="human",
           edge="gneE35",
           vehs_per_hour=5000,
            depart_lane="random",
            depart_speed="random",
            color="blue")


file_dir = "/home/llavezzo/"
net_params = NetParams(
    template="/mnt/c/Users/llave/Documents/GitHub/flow_osuphysics/lucalavezzo/constructionV4.net.xml",
    inflows=inflow
)

flow_params = dict(
    # name of the experiment
    exp_tag="construction_traffic",

    # name of the flow environment the experiment is running on
    env_name=myEnv,  # <------ here we replace the environment with our new environment

    # name of the network class the experiment is running on
    network=Network,

    # simulator that is used by the experiment
    simulator='traci',

    # sumo-related parameters (see flow.core.params.SumoParams)
    sim=SumoParams(
        sim_step=0.1,
        render=False,
        restart_instance=True
    ),

    # environment related parameters (see flow.core.params.EnvParams)
    env=EnvParams(
        horizon=HORIZON,
        warmup_steps=750,
        clip_actions=False,
        additional_params={
            "target_velocity": 20,
            "sort_vehicles": False,
            "max_accel": 1,
            "max_decel": 1,
        },
    ),

    # network-related parameters (see flow.core.params.NetParams and the
    # network's documentation or ADDITIONAL_NET_PARAMS component)
    net=net_params,

    # vehicles to be placed in the network at the start of a rollout (see
    # flow.core.params.VehicleParams)
    veh=vehicles,

    # parameters specifying the positioning of vehicles upon initialization/
    # reset (see flow.core.params.InitialConfig)
    initial=initial_config
)


def setup_exps():
    """Return the relevant components of an RLlib experiment.

    Returns
    -------
    str
        name of the training algorithm
    str
        name of the gym environment to be trained
    dict
        training configuration parameters
    """
    alg_run = "PPO"

    agent_cls = get_agent_class(alg_run)
    config = agent_cls._default_config.copy()
    config["num_workers"] = N_CPUS
    config["train_batch_size"] = HORIZON * N_ROLLOUTS
    config["gamma"] = 0.999  # discount rate
    config["model"].update({"fcnet_hiddens": [3, 3]})
    config["use_gae"] = True
    config["lambda"] = 0.97
    config["kl_target"] = 0.02
    config["num_sgd_iter"] = 10
    config['clip_actions'] = False  # FIXME(ev) temporary ray bug
    config["horizon"] = HORIZON

    # save the flow params for replay
    flow_json = json.dumps(
        flow_params, cls=FlowParamsEncoder, sort_keys=True, indent=4)
    config['env_config']['flow_params'] = flow_json
    config['env_config']['run'] = alg_run

    create_env, gym_name = make_create_env(params=flow_params, version=0)

    # Register as rllib env
    register_env(gym_name, create_env)
    return alg_run, gym_name, config


/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

In [ ]:
alg_run, gym_name, config = setup_exps()
ray.init(num_cpus=N_CPUS + 1)
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,
        "env": gym_name,
        "config": {
            **config
        },
        "checkpoint_freq": 20,
        "checkpoint_at_end": True,
        "max_failures": 999,
        "stop": {
            "training_iteration": 200,
        },
    }
})


2020-03-06 14:26:08,202	WARNING services.py:592 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-03-06 14:26:08,209	INFO resource_spec.py:212 -- Starting Ray with 5.13 GiB memory available for workers and up to 2.57 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-06 14:26:08,794	WARNING services.py:1080 -- Failed to start the dashboard. The dashboard requires Python 3 as well as 'pip install aiohttp psutil setproctitle grpcio'.


Trial name,status,loc
PPO_myEnv-v0_557c677c,RUNNING,


(pid=1483) E0306 14:26:11.071015000    1483 socket_utils_common_posix.cc:208] check for SO_REUSEPORT: {"created":"@1583522771.071003600","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":185,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(pid=1483) E0306 14:26:11.071185400    1483 socket_utils_common_posix.cc:313] setsockopt(TCP_USER_TIMEOUT) Protocol not available
(pid=1484) E0306 14:26:11.050134600    1484 socket_utils_common_posix.cc:208] check for SO_REUSEPORT: {"created":"@1583522771.050111500","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":185,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(pid=1484) E0306 14:26:11.050520100    1484 socket_utils_common_posix.cc:313] setsockopt(TCP_USER_TIMEOUT) Protocol not available
(pid=raylet)

(pid=1483) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=1483)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=1483) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=1483)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=1483) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=1483)   _np_qint16 = np.dtype([("qint16", np.int16, 1)]

(pid=1482) Warning: Vehicle 'flow_00.23' performs emergency braking with decel=-86.11 wished=4.50 severity=18.13, time=120.20.
(pid=1482) Warning: Vehicle 'flow_10.35' performs emergency braking with decel=-128.54 wished=4.50 severity=27.57, time=122.20.
(pid=1482) Warning: Vehicle 'flow_10.25' performs emergency braking with decel=-9.92 wished=4.50 severity=1.21, time=137.60.
(pid=1483) Warning: Vehicle 'flow_00.20' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=142.40.
(pid=1483) Warning: Teleporting vehicle 'flow_00.20'; collision with vehicle 'flow_00.15', lane=':gneJ14_5_0', gap=-1.00, time=144.10 stage=move.
(pid=1483) Warning: Vehicle 'flow_00.20' ends teleporting on edge 'gneE13', time 144.10.
(pid=1482) Warning: Vehicle 'flow_10.28' performs emergency braking with decel=-118.68 wished=4.50 severity=25.37, time=158.60.
(pid=1482) Warning: Vehicle 'flow_10.23' performs emergency braking with decel=-9.84 wished=4.50 severity=1.19, time=158.70.
(pid=14

...

Trial name,status,loc,iter,total time (s),timesteps,reward
PPO_myEnv-v0_557c677c,RUNNING,172.31.50.197:1484,1,963.155,30000,13613.3


(pid=1484) 2020-03-06 14:42:28,500	WARNING ppo.py:140 -- The magnitude of your environment rewards are more than 1361.0x the scale of `vf_clip_param`. This means that it will take more than 1361.0 iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.
(pid=1483) Warning: Vehicle 'flow_00.11' performs emergency braking with decel=-93.62 wished=4.50 severity=19.81, time=80.40.
(pid=1483) Warning: Vehicle 'flow_00.12' performs emergency braking with decel=-23.95 wished=4.50 severity=4.32, time=81.50.
(pid=1483) Warning: Vehicle 'flow_00.12' performs emergency braking with decel=-43.13 wished=4.50 severity=8.59, time=82.30.
(pid=1483) Warning: Vehicle 'flow_10.8' performs emergency braking with decel=-10.70 wished=4.50 severity=1.38, time=85.50.
(pid=1482) Warning: Vehicle 'flow_10.5' performs emergency braking with decel=-30.92 wished=4.50 severity=5.87, time=87.80.
(pid=1482) Warning: Vehicle 'flow_00.12' performs emergency braking w

(pid=1482) Warning: Vehicle 'flow_10.56' performs emergency braking with decel=-13.21 wished=4.50 severity=1.94, time=198.80.
(pid=1482) Warning: Vehicle 'flow_00.30' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=199.10.
(pid=1482) Warning: Vehicle 'flow_10.56' performs emergency braking with decel=-75.65 wished=4.50 severity=15.81, time=201.80.
(pid=1482) Warning: Vehicle 'flow_10.63' performs emergency braking with decel=-104.70 wished=4.50 severity=22.27, time=207.40.
(pid=1482) Warning: Vehicle 'flow_10.60' performs emergency braking with decel=-14.31 wished=4.50 severity=2.18, time=207.50.
(pid=1482) Warning: Vehicle 'flow_10.62' performs emergency braking with decel=-77.31 wished=4.50 severity=16.18, time=208.80.
(pid=1482) Warning: Vehicle 'flow_10.64' performs emergency braking with decel=-19.56 wished=4.50 severity=3.35, time=210.20.
(pid=1482) Warning: Vehicle 'flow_00.30' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, tim

(pid=1482) Warning: Vehicle 'flow_10.20' performs emergency braking with decel=-58.15 wished=4.50 severity=11.92, time=150.90.
(pid=1482) Warning: Vehicle 'flow_10.21' performs emergency braking with decel=-90.80 wished=4.50 severity=19.18, time=152.40.
(pid=1482) Warning: Vehicle 'flow_10.25' performs emergency braking with decel=-14.26 wished=4.50 severity=2.17, time=152.50.
(pid=1483) Warning: Vehicle 'flow_10.24' performs emergency braking with decel=-78.23 wished=4.50 severity=16.38, time=159.40.
(pid=1483) Warning: Vehicle 'flow_10.19' performs emergency braking with decel=-129.32 wished=4.50 severity=27.74, time=160.60.
(pid=1482) Warning: Vehicle 'flow_10.14' performs emergency braking with decel=-127.95 wished=4.50 severity=27.43, time=156.00.


2020-03-06 15:13:03,371	ERROR worker.py:1065 -- listen_error_messages_raylet: Error 113 connecting to 172.31.50.197:25715. No route to host.
2020-03-06 15:13:03,386	ERROR worker.py:965 -- print_logs: Error 113 connecting to 172.31.50.197:25715. No route to host.
2020-03-06 15:13:03,387	ERROR import_thread.py:93 -- ImportThread: Error 113 connecting to 172.31.50.197:25715. No route to host.
2020-03-06 15:14:43,530	ERROR trial_runner.py:480 -- Trial PPO_myEnv-v0_557c677c: Error processing event.
Traceback (most recent call last):
  File "/home/llavezzo/anaconda2/envs/flow/lib/python3.6/site-packages/ray/tune/trial_runner.py", line 424, in _process_trial
    result = self.trial_executor.fetch_result(trial)
  File "/home/llavezzo/anaconda2/envs/flow/lib/python3.6/site-packages/ray/tune/ray_trial_executor.py", line 377, in fetch_result
    result = ray.get(trial_future[0], DEFAULT_GET_TIMEOUT)
  File "/home/llavezzo/anaconda2/envs/flow/lib/python3.6/site-packages/ray/worker.py", line 1494, 